In [14]:
#====================================
# Author: gmd8si
# Date: 16, November, 2016
#====================================
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
from detector_dr import Detector
from util import load_image           # Questionalable since we can use our own loading pipeline
import os
import matplotlib.pyplot as plt
import csv
#================================================
# Setting Hyper-Parameters 
#================================================

#weight_path = '../data/caffe_layers_value.pickle'
#model_path = '../models/caltech256/'
PRETRAINED_MODEL_PATH= None #'../models/caltech256/model-0'
N_EPOCHS = 10000
INIT_LEARNING_RATE = 0.01
WEIGHT_DECAY_RATE = 0.0005
MOMENTUM = 0.9
IMAGE_HEIGHT  = 512    #960
IMAGE_WIDTH   = 512    #720
NUM_CHANNELS  = 3
BATCH_SIZE = 50
N_LABELS = 5
DROPOUT = 0.50  
LOGS_PATH = './tensorflow_logs/new'
WEIGHT_PATH = None
TRAINSET_PATH = '/home/gmd8si/Weakly_detector/data/kaggle512_train1000.csv'
TESTSET_PATH ='/home/gmd8si/Weakly_detector/data/kaggle512_test.csv'

# Reading Data

In [23]:
#=======================================================================================================
# Reading Training data from CSV FILE
#=======================================================================================================
# load csv content, reading labels and filepaths
#csv_path = tf.train.string_input_producer(['/home/gmd8si/Weakly_detector/data/kaggle512_train.csv'])
#def load_data(data_path, batch_size):
csv_path = tf.train.string_input_producer([TRAINSET_PATH], num_epochs=1, shuffle=True, capacity = 60)
textReader = tf.TextLineReader()
_, csv_content = textReader.read(csv_path)
im_name, im_label = tf.decode_csv(csv_content, record_defaults=[[""], [1]])

# load images, and convert labels into one_hot encoded form
# DO WE NEED HOT-ENCODED VECTORS

im_content = tf.read_file(im_name)
train_image = tf.image.decode_jpeg(im_content, channels=3)
train_image = tf.cast(train_image, tf.float32) / 255. # could be unnecessary
#image = image.set_shape([IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS])
size = tf.cast([512, 512], tf.int32)
train_image = tf.image.resize_images(train_image, size)
#label = tf.one_hot(label, 5, 1, 0 )
train_label = tf.cast(im_label, tf.int64) # unnecessary
# Make Batches of images with shuffling
#min_after_dequeue = 60                                               # Defines how big a buffer we will randomly sample from -- bigger means better shuffling but slower start up and more memory used
#capacity = min_after_dequeue + 3 * BATCH_SIZE                         # Capacity must be larger than min_after_dequeue and the amount larger determines the maximum we will prefetch.
train_image_batch, train_label_batch = tf.train.batch([train_image, train_label], batch_size=BATCH_SIZE)

In [24]:
#=======================================================================================================
# Reading Test data from CSV FILE
#=======================================================================================================

test_csv_path = tf.train.string_input_producer([TESTSET_PATH], shuffle=True) # No need to set no. epochs here
test_textReader = tf.TextLineReader()
_, test_content = test_textReader.read(test_csv_path)
test_image, test_label = tf.decode_csv(test_content, record_defaults=[[""], [1]])


test_image_content = tf.read_file(test_image)
test_image = tf.image.decode_jpeg(test_image_content, channels=3)
test_image = tf.cast(test_image, tf.float32) / 255. # could be unnecessary
#image = image.set_shape([IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS])
size = tf.cast([512, 512], tf.int32)
test_image = tf.image.resize_images(test_image, size)
#label = tf.one_hot(label, 5, 1, 0 )
test_label = tf.cast(test_label, tf.int64) # unnecessary
# Make Batches of images with shuffling
#min_after_dequeue = 60                                               # Defines how big a buffer we will randomly sample from -- bigger means better shuffling but slower start up and more memory used
#capacity = min_after_dequeue + 3 * BATCH_SIZE                         # Capacity must be larger than min_after_dequeue and the amount larger determines the maximum we will prefetch.
test_image_batch, test_label_batch = tf.train.batch([test_image, test_label], batch_size=BATCH_SIZE)

# Determining datasets length

In [15]:
trainfile = open(TRAINSET_PATH, "rb")
reader = csv.reader(trainfile)
trainset = sum(1 for row in reader)
print trainset
trainfile.close()

testfile = open(TESTSET_PATH, "rb")
reader = csv.reader(testfile)
testset = sum(1 for row in reader)
print testset
testfile.close()

1002
48573


# Placeholder

In [25]:
learning_rate = tf.placeholder( tf.float32, [])
images_tf = tf.placeholder( tf.float32, [None, 512, 512, 3], name="images")
labels_tf = tf.placeholder( tf.int64, [None], name='labels')  # the dimensions could be [None,N_CLASSES]

# Optimizer Properties

In [5]:
detector = Detector(N_LABELS)

In [6]:
p1,p2,p3,p4,conv5, conv6, gap, output = detector.inference(images_tf)
loss_tf = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits( output, labels_tf ))

In [7]:
weights_only = filter( lambda x: x.name.endswith('W:0'), tf.trainable_variables() )
weight_decay = tf.reduce_sum(tf.pack([tf.nn.l2_loss(x) for x in weights_only])) * WEIGHT_DECAY_RATE
loss_tf += weight_decay

In [11]:
optimizer = tf.train.MomentumOptimizer( learning_rate, MOMENTUM)
grads_and_vars = optimizer.compute_gradients( loss_tf )
grads_and_vars = map(lambda gv: (gv[0], gv[1]) if ('conv6' in gv[1].name or 'GAP' in gv[1].name) else (gv[0]*0.1, gv[1]), 
                     grads_and_vars)
#grads_and_vars = [(tf.clip_by_value(gv[0], -5., 5.), gv[1]) for gv in grads_and_vars]
train_op = optimizer.apply_gradients( grads_and_vars )

# Accuracy

In [26]:
#label_predictions = output.argmax(axis=1)
#accuracy = (label_predictions == labels_tf).sum()
# check if we have access to labels_tf in session, like the train.caltech

correct_pred = tf.equal(tf.argmax(output, 1), labels_tf)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Tensorflow checkpoint saver

In [16]:
# Saver Operation to save and restore all variables, first create directory
ckpt_dir = "./ckpt_dir"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
saver = tf.train.Saver(max_to_keep=50)   #Indicates max number of recent checkpoint files to keep

In [17]:
if PRETRAINED_MODEL_PATH:
    print "Pretrained"
    saver.restore(sess, PRETRAINED_MODEL_PATH)

In [18]:
if not os.path.exists('./results'):
    os.makedirs('./results')
f_log = open('./results/DR_1_vggCAM.txt', 'w')

iterations = 0
loss_list = []

# Training and Testing Sessions

In [27]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())    
    sess.run(tf.initialize_local_variables())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for epoch in range(2):
    
        try: 
            #while not coord.should_stop():
            for i in range (25): # This loop should be bigger than 1 epoch size
                train_imbatch, train_labatch = sess.run([train_image_batch, train_label_batch])
                _, loss_val, output_val, acc = sess.run([train_op, loss_tf, output, accuracy], feed_dict={
                        learning_rate: INIT_LEARNING_RATE, images_tf: train_imbatch, labels_tf: train_labatch})
                
                loss_list.append(loss_val)                              # This must be for visualizing loss curve 
                iterations += 1
                print iterations
                if iterations % 5 == 0:   # after 5 batches
                    print "======================================"
                    print "Epoch", epoch, "Iteration", iterations
                    print "Processed", (iterations*BATCH_SIZE), '/', trainset

                    #label_predictions = output_val.argmax(axis=1)
                    #acc = (label_predictions == current_labels).sum()

                    print "Accuracy:", acc  #, '/', len(current_labels)
                    #print "Accuracy Percentage:", (acc/len(current_labels))*100
                    print "Training Loss:", np.mean(loss_list)
                    print "\n"
                    loss_list = []                
                
        except tf.errors.OutOfRangeError:
            print ('Epoch Reached: ', epoch)
            
        n_correct = 0
        n_data = 0
        
        test_imbatch, test_labatch = sess.run([test_image_batch, test_label_batch])
        output_vals, acc = sess.run([output, accuracy], feed_dict={images_tf:test_imbatch, labels_tf: test_labatch})

        #label_predictions = output_vals.argmax(axis=1)
        #acc = (label_predictions == current_labels).sum()

        n_correct += acc
        n_data += epoch*BATCH_SIZE

        acc_all = n_correct / float(n_data)
        f_log.write('epoch:'+str(epoch)+'\tacc:'+str(acc_all) + '\n')
        print "$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"
        print 'epoch:'+str(epoch)+'\tacc:'+str(acc_all) + '\n'
        print "$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"
        
        saver.save( sess, os.path.join( model_path, 'model'), global_step=epoch)
        INIT_LEARNING_RATE *= 0.99

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: Mean_3/_129 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1014_Mean_3", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Placeholder', defined at:
  File "/opt/dl/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/opt/dl/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/gmd8si/tf/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/gmd8si/tf/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in 